In [13]:
#code for renaming
import os

# Function to delete .DS_Store file if present
def delete_ds_store_in_folders(path):
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            ds_store_file = os.path.join(folder_path, '.DS_Store')
            if os.path.exists(ds_store_file):
                os.remove(ds_store_file)


# Function to rename folders (not being used)
def rename_folders(path):
    for folder_name in os.listdir(path):
        if "(" in folder_name:
            new_folder_name = folder_name.split("(")[0].strip()
            os.rename(os.path.join(path, folder_name), os.path.join(path, new_folder_name))


# Function to rename images with order numbers and folder name
def rename_images_with_order(path):
    for folder_name in os.listdir(path):
        folder_path = os.path.join(path, folder_name)
        if os.path.isdir(folder_path):
            # Sort images to ensure consistent order
            images = sorted(os.listdir(folder_path))
            for i, image_name in enumerate(images, 1):  # Start numbering from 1
                file_ext = os.path.splitext(image_name)[-1]  # Get the file extension
                new_image_name = f"{i}_{folder_name}{file_ext}"
                os.rename(os.path.join(folder_path, image_name), os.path.join(folder_path, new_image_name))


# Specify the path to the main folder containing subfolders and images
main_folder_path = r'C:\Users\76543\Downloads\input'

# Delete .DS_Store file if present in folders
delete_ds_store_in_folders(main_folder_path)

# Rename images with order numbers and folder name
rename_images_with_order(main_folder_path)

In [11]:
#code for optimisation

import os
from PIL import Image, ImageFilter, ImageEnhance

def reduce_image_size(input_image_path, output_image_path, quality, scale_factor, sharpen_params, contrast, color_enhance, noise_reduction):
    try:
        with Image.open(input_image_path) as img:
            original_format = img.format

            if original_format in ['JPEG', 'JPG']:
                img = img.convert('RGB')

            width, height = img.size
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)
            img = img.resize((new_width, new_height), Image.LANCZOS)

            img = img.filter(ImageFilter.UnsharpMask(radius=sharpen_params["radius"], percent=sharpen_params['percent'], threshold=sharpen_params['threshold']))

            if contrast:
                enhancer = ImageEnhance.Contrast(img)
                img = enhancer.enhance(contrast)

            if color_enhance:
                enhancer = ImageEnhance.Color(img)
                img = enhancer.enhance(color_enhance)
            else:
                # Optional adjustment if images appear too dark
                enhancer = ImageEnhance.Brightness(img)
                img = enhancer.enhance(1.2)  # Increase brightness slightly

            if noise_reduction:
                img = img.filter(ImageFilter.MedianFilter(size=1))

            img.save(output_image_path, quality=quality, optimize=True, progressive=True)
            print(f"Processed image saved as {output_image_path}")

    except Exception as e:
        print(f"Error processing {input_image_path}: {e}")

def get_file_size(file_path):
    return os.path.getsize(file_path) / 1024  # size in KB

def adjust_parameters_and_process(input_image_path, output_image_path, initial_quality, initial_scale_factor, min_size_kb, max_size_kb, sharpen_params, contrast, color_enhance, noise_reduction):
    quality = initial_quality
    scale_factor = initial_scale_factor

    while True:
        reduce_image_size(input_image_path, output_image_path, quality, scale_factor, sharpen_params, contrast, color_enhance, noise_reduction)
       
        file_size_kb = get_file_size(output_image_path)

        if min_size_kb <= file_size_kb <= max_size_kb:
            print(f"Image size is within range: {file_size_kb:.2f} KB")
            break
        else:
            print(f"Image size out of range: {file_size_kb:.2f} KB")
            # Adjust parameters
            if file_size_kb < min_size_kb:
                scale_factor *= 1.05  # Increase scale factor by 0.05
            else:
                scale_factor *= 0.95  # Decrease scale factor by 0.05

            # Ensure scale_factor stays within reasonable bounds
            if scale_factor < 0.1:
                scale_factor = 0.1
            elif scale_factor > 2.0:
                scale_factor = 2.0

            # Break if scale_factor adjustments are exhausted
            if scale_factor <= 0.1 or scale_factor >= 2.0:
                print("Unable to achieve desired file size with current scale_factor adjustments.")
                break

input_folder = r'C:\Users\76543\Downloads\input'  # Update with your folder path
output_folder = r'C:\Users\76543\Downloads\output'  # Update with your folder path
initial_quality = 80
initial_scale_factor = 0.5
min_size_kb = 65
max_size_kb = 80
sharpen_params = {'radius': 1, 'percent': 50, 'threshold': 3}
contrast = 1.1
color_enhance = 1.0  # Set to 1.0 to avoid color darkening
noise_reduction = True

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List of accepted image extensions
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')

# Iterate over subfolders and process images
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    if os.path.isdir(subfolder_path):
        output_subfolder_path = os.path.join(output_folder, subfolder)
        if not os.path.exists(output_subfolder_path):
            os.makedirs(output_subfolder_path)

        for filename in os.listdir(subfolder_path):
            if filename.startswith('.'):  # Skip hidden files
                continue

            if not filename.lower().endswith(valid_extensions):
                print(f"Skipping non-image file: {filename}")
                continue

            input_image_path = os.path.join(subfolder_path, filename)
            output_image_path = os.path.join(output_subfolder_path, filename)

            try:
                adjust_parameters_and_process(input_image_path, output_image_path, initial_quality, initial_scale_factor, min_size_kb, max_size_kb, sharpen_params, contrast, color_enhance, noise_reduction)
            except Exception as e:
                print(f"Error processing {filename}: {e}")

Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 143.89 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 130.75 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 119.26 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 108.71 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 98.07 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 90.63 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size out of range: 82.65 KB
Processed image saved as C:\Users\76543\Downloads\output\1\1_1.jpeg
Image size is within range: 75.29 KB
Processed image saved as C:\Users\76543\Downloads\output\2\1_2.png
Image size out of range: 270.56 KB
Processed image saved as C:\Users\76543\Downloads\output\2\1_2.png
Image si

In [17]:
import os
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

def resize_image(image, desired_width, desired_height):
    original_width, original_height = image.size
   
    # Calculate aspect ratio
    aspect_ratio = original_width / original_height
   
    # Determine new dimensions to fit within the desired dimensions
    if original_width / desired_width > original_height / desired_height:
        new_width = desired_width
        new_height = int(desired_width / aspect_ratio)
    else:
        new_height = desired_height
        new_width = int(desired_height * aspect_ratio)
   
    # Resize the image
    image = image.resize((new_width, new_height), Image.LANCZOS)
   
    return image

def add_border_with_dimensions(image_path, output_path, desired_width, desired_height, border_color=(255, 255, 255)):
    try:
        # Load the image
        original_image = Image.open(image_path)
       
        # Resize the image to fit within the desired dimensions
        resized_image = resize_image(original_image, desired_width, desired_height)
        resized_width, resized_height = resized_image.size

        # Create a new image with the desired dimensions and the border color
        new_image = Image.new("RGB", (desired_width, desired_height), border_color)

        # Calculate the position to paste the resized image to center it
        paste_x = (desired_width - resized_width) // 2
        paste_y = (desired_height - resized_height) // 2

        # Paste the resized image onto the new image
        new_image.paste(resized_image, (paste_x, paste_y))

        # Save the new image
        new_image.save(output_path)
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

def process_images(input_directory, output_directory, desired_width, desired_height):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through all files and subfolders
    for root, _, files in os.walk(input_directory):
        # Create the corresponding output subfolder
        relative_path = os.path.relpath(root, input_directory)
        output_subfolder = os.path.join(output_directory, relative_path)
        os.makedirs(output_subfolder, exist_ok=True)

        # Process each image file in the current folder
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                input_path = os.path.join(root, file)
                output_path = os.path.join(output_subfolder, f"bordered_{file}")
                add_border_with_dimensions(input_path, output_path, desired_width, desired_height)

# Example usage
input_directory = r'C:\Users\76543\Downloads\output'
output_directory = r'C:\Users\76543\Downloads\final_output'
desired_width = 500  # Desired width
desired_height = 600  # Desired height

process_images(input_directory, output_directory, desired_width, desired_height)